In [2]:
import os
import numpy as np
import cv2

from PIL import Image
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [35]:
# 加载数据集
datas = []
labels = []

# 加载1 样本
img_dir1 = '../datas/split_images/1'
image_list = os.listdir(img_dir1)
for name in image_list:
    if '.jpg' in name:
        path = os.path.join(img_dir1, name)
        image = cv2.imread(path)    # rgb方式读取图片
        img = cv2.cvtColor(image, cv2.COLOR_RGB2BGRA)  # 灰度处理

        datas.append(img.reshape(-1))
        labels.append(name[-6:-4])
#         s = name[-5:-4]
#         if s == '-':
#             labels.append(0)
#         elif s =='+':
#             labels.append(1)
#         else:
#             print(s)

In [39]:
print(len(datas))
print(set(labels))

11256
{'53', '65', '42', '58', '93', '51', '34', '87', '90', '20', '82', '52', '61', '39', '21', '97', '43', '50', '66', '56', '28', '55', '27', '84', '80', '15', '74', '49', '63', '91', '29', '54', '71', '40', '19', '62', '31', '72', '35', '64', '23', '95', '45', '68', '79', '30', '22', '41', '86', '37', '47', '46', '18', '78', '88', '60', '11', '25', '99', '36', '75', '59', '57', '24', '44', '10', '69', '17', '32', '94', '77', '16', '83', '96', '33', '70', '98', '92', '14', '26', '12', '48', '76', '85', '81', '38', '67', '89', '13', '73'}


In [42]:
# 样本转化
X = np.array(datas) / 255
# Y = LabelBinarizer().fit_transform(labels)
Y = np.array(labels)

# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=9)

In [48]:
# 构建模型训练数据
# 1 最佳参数： {'clf__C': 2, 'clf__gamma': 0.01}
# 2 最佳参数： {'clf__C': 1, 'clf__gamma': 0.05}
# 3 最佳参数 ：{'clf__C': 2, 'clf__gamma': 0.01}
clf = SVC(gamma=0.01, C=2, random_state=9)
clf.fit(x_train, y_train)

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=9, shrinking=True, tol=0.001,
    verbose=False)

In [49]:
# 训练效果
print(f'预测得分：{clf.score(x_test, y_test) * 100:.2f}%')

预测得分：92.98%


In [50]:
print(clf.predict(x_test[40:60]))
print(y_test[40:60])

['98' '95' '76' '23' '87' '49' '43' '96' '70' '92' '76' '47' '59' '45'
 '16' '78' '38' '93' '88' '75']
['98' '95' '76' '23' '87' '49' '43' '96' '70' '92' '76' '47' '59' '45'
 '16' '78' '32' '95' '88' '75']


In [8]:
# 交叉验证，寻找最佳参数
models = [
    Pipeline([
        ('clf', SVC())
    ])
]

parmaters = {
    'clf__C': [0.01, 0.05, 0.1, 1, 1.5, 2],
    'clf__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1]
}

In [46]:
model = GridSearchCV(models[0], param_grid=parmaters, n_jobs=8, cv=3)
model.fit(x_train[:1000], y_train[:1000])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('clf',
                                        SVC(C=1.0, cache_size=200,
                                            class_weight=None, coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='auto_deprecated',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='warn', n_jobs=8,
             param_grid={'clf__C': [0.01, 0.05, 0.1, 1, 1.5, 2],
                         'clf__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1]},
             pre_dispatch='2*n_jobs', refit=True, ret

In [47]:
print(model.best_params_)

{'clf__C': 2, 'clf__gamma': 0.01}


In [51]:
# 保存模型
joblib.dump(clf, 'clf_1.model')

['clf_1.model']